In [ ]:
from dotenv import load_dotenv
from langchain.schema import Document
from operator import itemgetter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.chains.query_constructor.base import AttributeInfo
from langchain_community.document_transformers import LongContextReorder
from langchain.retrievers import SelfQueryRetriever
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.runnables import (
    RunnableLambda,
    ConfigurableField,
)

from langchain_core.prompts import PromptTemplate
from langchain_text_splitters import (
    RecursiveJsonSplitter,
)
import json

## Without Metadata

In [ ]:
#전처리된 데이터 10개 약 20만 7천개
#마지막 파일빼고는 24000개씩 데이터 적재
load_dotenv()
for i in range(1, 10):
    print(i)
    with open(f'./data/processed_merged_data{i}.json', 'r',  encoding='utf-8') as j:
        data = json.load(j)
    json_splitter = RecursiveJsonSplitter(max_chunk_size=1300)
    documents = json_splitter.create_documents(texts=data, ensure_ascii=False)
    db = Chroma.from_documents(
        documents=documents,
        embedding=OpenAIEmbeddings(model='text-embedding-3-small', show_progress_bar=True),
        collection_name='books',
        persist_directory='./db'
    )
    

## With Metadata

In [ ]:
load_dotenv()
for i in range(1, 10) :
    print(i)
    with open(f'./data/processed_merged_data{i}.json') as f:
        data = json.load(f)
        tmp_documents = []
        for content_data in data:
            page_content = content_data.get("content", "")  # 예시로 'description' 필드를 page_content로 사용
            metadata = {
                "title": content_data.get("title", ""),
                "creator": content_data.get("creator", ""),
                "publisher": content_data.get("publisher", ""),
                "issued": content_data.get("issued", ""),
                "location": content_data.get("location", ""),
                "mainCategoryDescription": content_data.get("mainCategoryDescription", ""),
                "alternative": content_data.get("alternative", ""),
                "keyword": content_data.get("keyword", ""),
                "seeAlso": content_data.get("seeAlso", "")
            }
            
            # Document 객체 생성 및 추가
            tmp_documents.append(Document(page_content=page_content, metadata=metadata))
        
        # persist_directory 하면 자동으로 축적됨
        vectordb = Chroma.from_documents(
            documents=tmp_documents,
            collection_name='books_w_meta',
            persist_directory='./db_meta',
            embedding=OpenAIEmbeddings(model="text-embedding-3-small", show_progress_bar=True)
        )      


In [ ]:
vectordb = Chroma(collection_name='books_w_meta', persist_directory='./db_meta', client=client, collection_metadata={"hnsw:M": 1024, "hnsw:ef": 512}, embedding_function=OpenAIEmbeddings(model="text-embedding-3-small", show_progress_bar=True))

In [ ]:
ret = vectordb.as_retriever(
    search_kwargs={
        'k': 1,               # 반환 무서 수
        'filter': {'creator': '김옥암'}    # 메타데이터 기반 필터링
    },
    metadata={
        "hnsw:space": "cosine", 
        "hnsw:construction_ef": 600, 
        "hnsw:search_ef": 100, 
        "hnsw:M": 60
    }
).configurable_fields(
    search_kwargs=ConfigurableField(
        id='search_kwargs',
        name='search_kwargs',
        description='search_kwargs값'
    ),
    metadata=ConfigurableField(
        id='metadata',
        name='metadata',
        description='metadata값'
    )
)

In [ ]:
ret.invoke(
    '교육에관한',
    config={'configurable': {
        'search_kwargs': {
            'k': 2,
            'filter': {'creator': '김옥암'}
        },
        'metadata': {
            "hnsw:space": "cosine", 
            "hnsw:construction_ef": 600, 
            "hnsw:search_ef": 1000, 
            "hnsw:M": 60
        }
    }})

In [ ]:
# 메타데이터 필드에 대한 정보를 정의합니다.
metadata_field_info = [
    AttributeInfo(
        name='title',  # 출시 연도를 나타내는 필드
        description='책 제목, 책',  # 필드 설명
        type='string',  # 데이터 타입은 정수
    ),
    AttributeInfo(
        name='creator',  # 상품 카테고리를 나타내는 필드
        description='작가',  # 필드 설명
        type='string',  # 데이터 타입은 문자열
    ),
    AttributeInfo(
        name='publisher',  # 사용자 평점을 나타내는 필드
        description='출판사 이름',  # 필드 설명
        type='string',  # 데이터 타입은 실수
    ),
    AttributeInfo(
        name='issued',  # 사용자 평점을 나타내는 필드
        description='출판 연도',  # 필드 설명
        type='string',  # 데이터 타입은 실수
    ),
    AttributeInfo(
        name='location',  # 사용자 평점을 나타내는 필드
        description='카테고리 값 1',  # 필드 설명
        type='string',  # 데이터 타입은 실수
    ),
    AttributeInfo(
        name='mainCategoryDescription',  # 사용자 평점을 나타내는 필드
        description='카테고리',  # 필드 설명
        type='string',  # 데이터 타입은 실수
    ),
    AttributeInfo(
        name='alternative',  # 사용자 평점을 나타내는 필드
        description='소제목 혹은 세부제목',  # 필드 설명
        type='string',  # 데이터 타입은 실수
    ),
    AttributeInfo(
        name='keyword',  # 사용자 평점을 나타내는 필드
        description='책을 설명하는 키워드',  # 필드 설명
        type='string',  # 데이터 타입은 실수
    ),
    AttributeInfo(
        name='seeAlso',  # 사용자 평점을 나타내는 필드
        description='책 검색 링크',  # 필드 설명
        type='string',  # 데이터 타입은 실수
    ),
    
]

In [ ]:
# SelfQueryRetriever 초기화: LLM과 벡터 저장소를 연결
retriever = SelfQueryRetriever.from_llm(
    llm=ChatOpenAI(model_name='gpt-4o-mini'),  # 사용할 언어 모델 설정
    vectorstore=vectordb,
    document_contents='책에 대한 기본 정보',  # 문서 내용 요약
    search_kwargs={'k': 3},  # 검색 결과 개수 설정
    metadata_field_info=metadata_field_info  # 메타데이터 필드 정보
)

In [ ]:
ret = vectordb.as_retriever(
    searcy_type='mmr',
    search_kwargs={
        'k': 20,               # 반환 무서 수
        'filter': {'creator': '이하중'}        # 메타데이터 기반 필터링
    }
)

In [ ]:
ret.get_relevant_documents('교육에 관한 내용')

In [ ]:
# 문서 재정렬 함수 정의
def reorder_documents(documents):
    # LongContextReorder 객체 생성: 긴 문맥을 재정렬하는 기능
    context_reorder = LongContextReorder()
    # 입력된 문서들을 재정렬
    documents_reordered = context_reorder.transform_documents(documents)
    
    # 사람이 볼 수 있도록 각 문서를 정제하여 문자열로 결합
    documents_formatted = '\n\n'.join([
        f"제목: {doc.metadata.get('title', 'N/A')}\n"
        f"작가: {doc.metadata.get('creator', 'N/A')}\n"
        f"출판사: {doc.metadata.get('publisher', 'N/A')}\n"
        f"발행일: {doc.metadata.get('issued', 'N/A')}\n"
        f"location: {doc.metadata.get('location', 'N/A')}\n"
        f"카테고리: {doc.metadata.get('mainCategoryDescription', 'N/A')}\n"
        f"대체 제목: {doc.metadata.get('alternative', 'N/A')}\n"
        f"키워드: {', '.join(doc.metadata.get('keyword', []))}\n"
        f"관련 작품: {', '.join(doc.metadata.get('seeAlso', []))}\n"
        f"내용: {doc.page_content}"
        for doc in documents_reordered
    ])
    print(documents_formatted)
    return documents_formatted  # 포맷팅된 문서 내용을 반환

# 프롬프트 템플릿 정의
template = '''
주어진 reference를 최대한 활용하라
{reference}

다음 질문에 답하라, reference에 없으면 "해당 정보를 찾을 수 없습니다."라고 알려줘
여러개의 작품을 추천할때 없는정보는 빼줘:
{question}

답변은 반드시 한국어로 해줘
'''

# 템플릿으로부터 프롬프트 생성
prompt = PromptTemplate.from_template(template)

# 모델과 파서 초기화
model = ChatOpenAI(model_name='gpt-4o-mini')
parser = StrOutputParser()

# chain 구성
chain = (
    {
        # 'reference' 키에 대해 여러 처리를 정의
        'reference': retriever  # SelfQueryRetriever를 통해 reference 추출
        | RunnableLambda(reorder_documents),  # 문서를 재정렬 및 포맷팅
        'question': itemgetter('question')  # 질문을 그대로 가져오기
    }
    | prompt  # 프롬프트 템플릿에 데이터 결합
    | model  # 모델에 프롬프트 전달하여 응답 생성
    | parser  # 모델의 응답을 파싱
)

In [ ]:
response = chain.invoke({
    'question': '작가의 이름이 황상순인 문서를 찾아줄래?'
})

print(response)


In [ ]:
response = chain.invoke({
    'question': '다다비주얼 출판사의 책을 알려줘'
})

print(response)
